# Carregamento do dataset

In [75]:
import pandas as pd

df = pd.read_csv("datasets/original.csv")
df = df.sort_values(by=["student_id", "period", "discipline"])

df.head(5)

,id,student_id,birthdate,sex,city,course,period,week_day,discipline,status,g1,g2,final_grade,class_skips
2138,1,99951,1992-05-21 02:00:00.000 -0300,F,Faxinal do Soturno,Ciências Contábeis,2021/1,Quarta,Matemática Aplicada - 60,Aprovado,7.5,7.0,7.2,0
5665,3,99951,1992-05-21 02:00:00.000 -0300,F,Faxinal do Soturno,Ciências Contábeis,2021/2,Segunda,Contabilidade Intermediária - 60,Aprovado,7.2,8.9,8.0,0
9160,2,99951,1992-05-21 02:00:00.000 -0300,F,Faxinal do Soturno,Ciências Contábeis,2022/1,Terça,Contabilidade Avançada - 60,Trancado,-1.0,-1.0,-1.0,0
28710,4,99955,1989-01-06 04:00:00.000 -0200,M,Faxinal do Soturno,Ciências Contábeis,2025/1,Segunda,Contabilidade Introdutória - 60,Aprovado,8.4,8.8,8.6,0
28312,8,99955,1989-01-06 04:00:00.000 -0200,M,Faxinal do Soturno,Ciências Contábeis,2025/1,Quarta,Legislação e Ética Profissional - 30,Aprovado,9.8,9.9,9.8,0


# Verificação do tamanho do dataset

In [76]:
original_df_len = len(df)
print(f"Quantidade de registros: {original_df_len}")

Quantidade de registros: 35995


# Pré processamento do dataset

In [77]:
# Realiza a limpeza da coluna "status"
df['status_clean'] = df['status'].astype(str).str.lower()
# Aplica o Label Encoding em "status"
df['status_encoded'] = df['status_clean'].astype('category').cat.codes


# Normalização das colunas de notas
def normalize_grade_column(colum_name: str):
    new_column_name = f"{ colum_name }_normalized"
    df[new_column_name] = pd.to_numeric(df[colum_name], errors='coerce') # valores inválidos viram NaN
    df[new_column_name] = (df[new_column_name] / 10).round(2) # Normalização e ajuste de decimais
    df.loc[df[new_column_name] == -0.1, new_column_name] = 0 # Valores "-0.1" trocados para 0

normalize_grade_column("g1")
normalize_grade_column("g2")
normalize_grade_column("final_grade")

# Tratamento da coluna "discipline"

# # Remoção do "traço" e do crédito da coluna disciplina no nome da mesma
df["discipline_normalized"] = df["discipline"].str.split(" - ").str[0].str.strip()
# Remoção para caso especial em "Gestão da Qualidade de Software"
df["discipline_normalized"] = df["discipline_normalized"].str.split(" -72").str[0].str.strip()

# Elimina do dataset disciplinas pouco cursadas

# Conta o número de alunos únicos em cada disciplina
discipline_counts = df.groupby("discipline")["student_id"].nunique()
# Identifica as disciplinas POPULARES (com MAIS alunos que o limiar)
popular_disciplines = discipline_counts[discipline_counts > 10].index
# Filtra o DataFrame original, mantendo APENAS as linhas que pertencem às disciplinas populares
df = df[df["discipline"].isin(popular_disciplines)]


# Criação de novas colunas binárias
df["is_approved"] = df["status_clean"].isin(["aprovado"]).astype(int)
df["canceled_discipline"] = df["status_clean"].isin(["trancado", "cancelado"]).astype(int)
df["skipped_discipline"] = df["status_clean"].isin(["reprovado por frequência"]).astype(int)


# Criação de "sex_normalized" para normalização do sexo biológico
df["sex_normalized"] = df["sex"].map({"M": 1, "F": 2}).fillna(0).astype(int)


# Criação de colunas normalizadas para "week_day", "course" e "period"
for col in ["week_day", "course", "period"]:
    df[f"{ col }_normalized"] = pd.factorize(df[col])[0] + 1


# Trata a coluna "birthdate"
df["birthdate_normalized"] = df["birthdate"].str.split(" ").str[0].str.strip()
# Converte essa string para um datetime "naive" (sem fuso)
naive_datetime = pd.to_datetime(df["birthdate_normalized"], errors='coerce')
# Define fuso horário GMT -3
df["birthdate_normalized"] = naive_datetime.dt.tz_localize('-03:00')


# Criação de "class_skips_normalized" para a normalização e ajuste das faltas por dia
df["class_skips_normalized"] = (df["class_skips"] / 4).astype(int)


# Realiza a normalização da coluna "discipline"
df['discipline_normalized'] = df['discipline_normalized'].astype('category').cat.codes


# Realiza a normalização da coluna "city"
df['city_normalized'] = df['city'].astype('category').cat.codes


# Salva dataset original com tratamentos
df.to_csv("datasets/original_and_treated.csv", index=False, encoding="utf-8")

df.head(5)

,id,student_id,birthdate,sex,city,course,period,week_day,discipline,status,...,is_approved,canceled_discipline,skipped_discipline,sex_normalized,week_day_normalized,course_normalized,period_normalized,birthdate_normalized,class_skips_normalized,city_normalized
2138,1,99951,1992-05-21 02:00:00.000 -0300,F,Faxinal do Soturno,Ciências Contábeis,2021/1,Quarta,Matemática Aplicada - 60,Aprovado,...,1,0,0,2,1,1,1,1992-05-21 00:00:00-03:00,0,38
5665,3,99951,1992-05-21 02:00:00.000 -0300,F,Faxinal do Soturno,Ciências Contábeis,2021/2,Segunda,Contabilidade Intermediária - 60,Aprovado,...,1,0,0,2,2,1,2,1992-05-21 00:00:00-03:00,0,38
9160,2,99951,1992-05-21 02:00:00.000 -0300,F,Faxinal do Soturno,Ciências Contábeis,2022/1,Terça,Contabilidade Avançada - 60,Trancado,...,0,1,0,2,3,1,3,1992-05-21 00:00:00-03:00,0,38
28710,4,99955,1989-01-06 04:00:00.000 -0200,M,Faxinal do Soturno,Ciências Contábeis,2025/1,Segunda,Contabilidade Introdutória - 60,Aprovado,...,1,0,0,1,2,1,4,1989-01-06 00:00:00-03:00,0,38
28312,8,99955,1989-01-06 04:00:00.000 -0200,M,Faxinal do Soturno,Ciências Contábeis,2025/1,Quarta,Legislação e Ética Profissional - 30,Aprovado,...,1,0,0,1,1,1,4,1989-01-06 00:00:00-03:00,0,38


# Verificação do tamanho do dataset

In [78]:
original_and_treated_df_len = len(df)
print(f"Quantidade de registros: {original_and_treated_df_len}")
print(f"Registros removidos durante o tratamento: {original_df_len - original_and_treated_df_len}")

Quantidade de registros: 35656
Registros removidos durante o tratamento: 339


# Geração de dataset final tratado

In [79]:
treated_df = df.copy(deep=True)

cols = [
    "id",
    "student_id",
    # "birthdate_normalized",
    "sex_normalized",
    "city_normalized",
    "course_normalized",
    "period_normalized",
    "week_day_normalized",
    "discipline_normalized",
    "status_encoded",
    "g1_normalized",
    "g2_normalized",
    "final_grade_normalized",
    "canceled_discipline",
    "skipped_discipline",
    "class_skips_normalized",
    "is_approved",
]

rename_map = {
    # "birthdate_normalized": "birthdate",
    "sex_normalized": "sex",
    "course_normalized": "course",
    "period_normalized": "period",
    "week_day_normalized": "week_day",
    "discipline_normalized": "discipline",
    "status_encoded": "status",
    "g1_normalized": "g1",
    "g2_normalized": "g2",
    "final_grade_normalized": "final_grade",
    "class_skips_normalized": "class_skips",
}

# Mantém somente colunas desejadas
treated_df = treated_df[cols]

# Realiza o renomeamento das colunas
treated_df = treated_df.rename(columns=rename_map)

treated_df.head(5)

,id,student_id,sex,city_normalized,course,period,week_day,discipline,status,g1,g2,final_grade,canceled_discipline,skipped_discipline,class_skips,is_approved
2138,1,99951,2,38,1,1,1,276,0,0.75,0.70,0.72,0,0,0,1
5665,3,99951,2,38,1,2,2,62,0,0.72,0.89,0.80,0,0,0,1
9160,2,99951,2,38,1,3,3,60,8,0.00,0.00,0.00,1,0,0,0
28710,4,99955,1,38,1,4,2,64,0,0.84,0.88,0.86,0,0,0,1
28312,8,99955,1,38,1,4,1,251,0,0.98,0.99,0.98,0,0,0,1


# Salvamento do dataset

In [80]:
treated_df.to_csv("datasets/original_treated.csv", index=False, encoding="utf-8")

# Remoção dos alunos aprovados em exame

Cerca de 2600 alunos possuem nota final menor que 7 (não há nota de exame), porém, estão aprovados, inviabilizando o treinamento dos modelos, portanto, estes serão removidos.

In [81]:
print("Total de registros no dataset pré limpeza de outliers EXAME", len(treated_df))

# Busca por alunos aprovados em exame
exam_students_filter = (treated_df['final_grade'] <= 0.7) & (treated_df['is_approved'] == 1)
not_approved_students = treated_df[exam_students_filter]

print(f"Total de alunos aprovados em exame: {len(not_approved_students)}")

# Remoção dos alunos aprovados em exame
treated_df = treated_df.drop(not_approved_students.index)
treated_df = treated_df.reset_index(drop=True)

print("Total de registros no dataset pós limpeza de outliers EXAME", len(treated_df))

Total de registros no dataset pré limpeza de outliers EXAME 35656
Total de alunos aprovados em exame: 2675
Total de registros no dataset pós limpeza de outliers EXAME 32981


# Equalização do dataset por aprovação e nota final

Cerca de 1300 alunos possuem nota final e estão devidamente reprovados com base na mesma frente a 24 mil aprovados com nota final, esses grupos serão equalizados no dataset.
1300 aprovados x 1300 reprovados

In [82]:
print("Total de registros no dataset pré equalização de dados", len(treated_df))

approved_students_filter = (treated_df['is_approved'] == 1) & (treated_df['final_grade'] > 0)
approved_students = treated_df[approved_students_filter]

not_approved_students_filter = (treated_df['is_approved'] == 0) & (treated_df['final_grade'] > 0)
not_approved_students = treated_df[not_approved_students_filter]

print(f"Total de alunos aprovados: {len(approved_students)}")
print(f"Total de alunos reprovados: {len(not_approved_students)}")

df_approved_balanced = approved_students.sample(n=1300, random_state=42)
df_not_approved_balanced = not_approved_students.sample(n=1300, random_state=42)

treated_df = pd.concat([df_approved_balanced, df_not_approved_balanced])
treated_df = treated_df.sample(frac=1, random_state=42).reset_index(drop=True)

print("Total de registros no dataset pós equalização de dados", len(treated_df))

Total de registros no dataset pré equalização de dados 32981
Total de alunos aprovados: 24467
Total de alunos reprovados: 1307
Total de registros no dataset pós equalização de dados 2600


In [84]:
treated_df.head(20)

,id,student_id,sex,city_normalized,course,period,week_day,discipline,status,g1,g2,final_grade,canceled_discipline,skipped_discipline,class_skips,is_approved
0,7208,100539,1,88,3,22,2,108,6,0.78,0.00,0.39,0,0,1,0
1,31459,102814,2,88,3,35,1,296,0,0.95,0.95,0.95,0,0,1,1
2,20305,101432,1,75,5,3,9,306,0,0.90,0.97,0.93,0,0,2,1
3,35118,103489,2,88,8,4,1,29,7,0.92,0.00,0.46,0,1,6,0
4,12084,100782,1,123,3,2,1,391,6,0.60,0.00,0.30,0,0,1,0
5,14708,100965,2,99,2,6,7,403,7,0.80,0.00,0.40,0,1,6,0
6,11941,100776,1,82,5,22,2,239,6,0.55,0.55,0.47,0,0,0,0
7,26899,102190,2,122,2,13,18,147,0,0.80,0.70,0.75,0,0,1,1
8,28651,102554,2,66,1,35,9,249,0,0.74,0.95,0.84,0,0,0,1
9,11300,100749,1,99,4,1,2,343,7,0.40,0.00,0.20,0,1,6,0
